# Day 5 
## Part 1
There are no loops in the map so it's just a question of working through the conversions.

In [1]:
from collections import defaultdict, namedtuple

Almanac = namedtuple("Almanac", "seeds map")
Range = namedtuple("Range", "source destination length")

def parse_data(s):
    blocks = s.strip().split("\n\n")
    
    seeds = [int(x) for x in blocks[0].split(": ")[1].split()]
    
    source_map = defaultdict(list)
    
    for block in blocks[1:]:
        lines = block.strip().splitlines()
        source, _, destination = lines[0].split()[0].split("-")
        for line in lines[1:]:
            destination_start, source_start, length = [int(x) for x in line.split()]
            source_map[(source, destination)].append(Range(source_start, destination_start, length))
            
    return Almanac(seeds, source_map)

test_data = parse_data("""seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4""")

test_data

Almanac(seeds=[79, 14, 55, 13], map=defaultdict(<class 'list'>, {('seed', 'soil'): [Range(source=98, destination=50, length=2), Range(source=50, destination=52, length=48)], ('soil', 'fertilizer'): [Range(source=15, destination=0, length=37), Range(source=52, destination=37, length=2), Range(source=0, destination=39, length=15)], ('fertilizer', 'water'): [Range(source=53, destination=49, length=8), Range(source=11, destination=0, length=42), Range(source=0, destination=42, length=7), Range(source=7, destination=57, length=4)], ('water', 'light'): [Range(source=18, destination=88, length=7), Range(source=25, destination=18, length=70)], ('light', 'temperature'): [Range(source=77, destination=45, length=23), Range(source=45, destination=81, length=19), Range(source=64, destination=68, length=13)], ('temperature', 'humidity'): [Range(source=69, destination=0, length=1), Range(source=0, destination=1, length=69)], ('humidity', 'location'): [Range(source=56, destination=60, length=37), Rang

In [2]:
def convert_number(number: int, ranges: [Range]) -> int:
    for r in ranges:
        if r.source <= number < r.source + r.length:
            return r.destination + number - r.source
    return number

def find_location(seed, almanac):
    source = "seed"
    current_number = seed
    
    while source != "location":
        destination = [
            d 
            for s, d in almanac.map 
            if s == source
        ][0]
        
        current_number = convert_number(
            current_number, 
            almanac.map[(source, destination)]
        )
        
        source = destination
        
    return current_number

[find_location(seed, test_data) for seed in test_data.seeds]

[82, 43, 86, 35]

In [3]:
def part_1(almanac):
    return min(find_location(seed, almanac) for seed in almanac.seeds)

assert part_1(test_data) == 35

In [4]:
data = parse_data(open("input").read())

part_1(data)

551761867

## Part 2
Applying the above to billions of seeds individually doesn't seem like a very good idea. 

Represent the resources as a list of slices, creating a new list of slices at each stage. 

At each stage, go through each range converting numbers within the range, and saving any missed for the other ranges. Any left are added unconverted for the next stage.

Use a `namedtuple` Slice rather than the unhashable built-in so repeats are eliminated.

In [5]:
Slice = namedtuple("Slice", "start stop")

def convert_numbers(numbers: [Slice], ranges: [Range]) -> [Slice]:
    new_numbers = set()
    numbers_to_process = numbers
    for r in ranges:
        numbers_remaining = set()
        for n in numbers_to_process:
            # Convert the overlapping range
            overlap_start = max(n.start, r.source)
            overlap_end = min(n.stop, r.source + r.length)
            if overlap_end > overlap_start:
                new_numbers.add(
                    Slice(
                        r.destination + (overlap_start - r.source),
                        (r.destination + (overlap_start - r.source) 
                             + (overlap_end - overlap_start))
                    )
                )
            # Still need to process anything before the start of the range
            if n.start < overlap_start:
                numbers_remaining.add(
                    Slice(
                        n.start,
                        min(n.stop, overlap_start)
                    )
                )
            # and after
            if n.stop > overlap_end:
                numbers_remaining.add(
                    Slice(
                        max(n.start, overlap_end),
                        n.stop
                    )
                )
        numbers_to_process = numbers_remaining
        
    return new_numbers | numbers_remaining

convert_numbers([Slice(0, 100)], [Range(40, 1000, 20)])

{Slice(start=0, stop=40),
 Slice(start=60, stop=100),
 Slice(start=1000, stop=1020)}

In [6]:
def find_locations(seed_start, seed_length, almanac):
    source = "seed"
    current_slices = {Slice(seed_start, seed_start + seed_length)}
    
    while source != "location":
        destination = [
            d 
            for s, d in almanac.map 
            if s == source
        ][0]
        
        current_slices = convert_numbers(
            current_slices, 
            almanac.map[(source, destination)]
        )
        
        source = destination
        
    return current_slices

min(s.start for s in find_locations(79, 14, test_data))

46

In [7]:
def part_2(almanac):
    locations = []
    for i in range(0, len(almanac.seeds), 2):
        seed_start = almanac.seeds[i]
        seed_length = almanac.seeds[i + 1]
        locations.append(
            min(
                s.start 
                for s in find_locations(seed_start, seed_length, almanac)
            )
        )
    return min(locations)

assert part_2(test_data) == 46   

In [8]:
part_2(data)

57451709